<a href="https://colab.research.google.com/github/Akarsh9120/SVMHybrid/blob/main/SVM_and_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.io import imread
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from skimage.util import random_noise

# Load and preprocess the data without text removal
Categories = ['cars', 'Ice cream cone', 'Cricket ball'] #the categories name should match with the folder names
image_size = (150, 150)
datadir = '/content/drive/MyDrive/Dataset' #Load all the datasets in the google drive if running on colab or any folder on the system and mention the relative path of it here..
images = []
targets = []

# Function to load and preprocess images
def preprocess_images(datadir, image_size):
    for category in Categories:
        path = os.path.join(datadir, category)
        if not os.path.exists(path):
            print(f"Warning: Directory '{path}' does not exist.")
            continue
        for img_name in os.listdir(path):
            if img_name.endswith('.png'):  # Filter out PNG images
                img_path = os.path.join(path, img_name)
                try:
                    img_array = imread(img_path)
                except Exception as e:
                    print(f"Error loading image '{img_path}': {e}")
                    continue

                # RGBA images to RGB
                if img_array.ndim == 3 and img_array.shape[2] == 4:
                    img_array = img_array[:, :, :3]  # Keep only RGB channels

                # to ensure image is in the correct format
                img_array = img_array.astype(np.uint8)

                img_resized = resize(img_array, image_size)
                if img_resized.shape == (150, 150, 3):  # To Check if the image has the correct dimensions
                    images.append(img_resized)
                    targets.append(Categories.index(category))
                else:
                    print(f"Ignored image {img_name} in category {category} due to incorrect dimensions: {img_resized.shape}")

# Load and preprocess images
preprocess_images(datadir, image_size)

# Convert lists to numpy arrays
images = np.array(images)
targets = np.array(targets)

if len(images) == 0:
    print("Error: No images were loaded. Please check the data directory.")
    exit()

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(images, targets, test_size=0.20, random_state=77, stratify=targets)

# add more random noise to images
def add_more_noise(images, var=0.1):
    noisy_images = []
    for image in images:
        noisy_image = random_noise(image, var=var)
        noisy_images.append(noisy_image)
    return np.array(noisy_images)

x_train_more_noisy = add_more_noise(x_train)
x_test_more_noisy = add_more_noise(x_test)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# CNN model architecture
model_cnn = Sequential()
model_cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model_cnn.add(MaxPooling2D((2, 2)))
model_cnn.add(Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(MaxPooling2D((2, 2)))
model_cnn.add(Conv2D(128, (3, 3), activation='relu'))
model_cnn.add(MaxPooling2D((2, 2)))
model_cnn.add(Conv2D(128, (3, 3), activation='relu'))
model_cnn.add(MaxPooling2D((2, 2)))
model_cnn.add(Flatten())
model_cnn.add(Dense(512, activation='relu'))
model_cnn.add(Dropout(0.5))
model_cnn.add(Dense(len(Categories), activation='softmax'))
model_cnn.compile(optimizer=Adam(lr=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# early stopping for CNN model
early_stopping_cnn = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train CNN model with early stopping and data augmentation on more noisy images
history_cnn_more_noisy = model_cnn.fit(datagen.flow(x_train_more_noisy, y_train, batch_size=32), epochs=25, validation_data=(x_test_more_noisy, y_test), callbacks=[early_stopping_cnn])

# Train SVM model using CNN features from more noisy images
features_train_more_noisy = model_cnn.predict(x_train_more_noisy)
features_test_more_noisy = model_cnn.predict(x_test_more_noisy)

# Adjust SVM hyperparameters
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [0.0001, 0.001, 0.1, 1], 'kernel': ['rbf', 'poly']}
svc = svm.SVC(probability=True)
model_svm = GridSearchCV(svc, param_grid, cv=5)
model_svm.fit(features_train_more_noisy, y_train)

# Combining both models for prediction
random_index = random.randint(0, len(x_test) - 1)
random_image = x_test[random_index]
random_image_feature = model_cnn.predict(np.expand_dims(random_image, axis=0))
svm_prediction = model_svm.predict(random_image_feature)
cnn_prediction = np.argmax(model_cnn.predict(np.expand_dims(random_image, axis=0)))

# Plot the image
plt.imshow(random_image)
plt.title('Original Image')
plt.show()

# Print the predicted image class for CNN and SVM
for i, category in enumerate(Categories):
    print(f"{category} =", random_image_feature[0][i] * 100, "%")
print("The predicted image is:", Categories[cnn_prediction])


#The execution time of this is 10 minutes please relax have a great day.
